<a href="https://colab.research.google.com/github/ptmhoang97/train_dataset/blob/main/eval_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Sep 13 09:18:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
batch_size_pipeline = 64
num_steps_training = 10000
learning_rate_pipeline = 0.08/100
warmup_learning_rate_pipeline = 0.026666/100
new_path = "/content/drive/MyDrive/workspaces_tensorflow/eval_model/attempt_20210913_122755_4"
models_drive_path = new_path[:new_path.rfind("/")]
model_name = new_path[new_path.rfind("/")+1:]
print(new_path)
print(models_drive_path)
print(model_name)

/content/drive/MyDrive/workspaces_tensorflow/eval_model/attempt_20210913_122755_4
/content/drive/MyDrive/workspaces_tensorflow/eval_model
attempt_20210913_122755_4


# 0. Setup Paths

In [40]:
import os

In [41]:
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [42]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc'),
    'MODEL_PATH': os.path.join(models_drive_path),
    'CHECKPOINT_PATH': os.path.join(new_path), 
    'OUTPUT_PATH': os.path.join(new_path, 'export'), 
    'TFJS_PATH':os.path.join(new_path, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join(new_path, 'tfliteexport'), 
 }

In [43]:
files = {
    'PIPELINE_CONFIG':os.path.join(new_path, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME),
}

In [44]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}
        print(path)

# 1. Install TF Object Dectection API

In [14]:
# https://www.tensorflow.org/install/source_windows

In [15]:
if os.name=='nt':
    !pip install wget
    import wget

In [16]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 62263, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 62263 (delta 3), reused 2 (delta 2), pack-reused 62259
Receiving objects: 100% (62263/62263), 574.37 MiB | 28.84 MiB/s, done.
Resolving deltas: 100% (43357/43357), done.


In [17]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Processing /content/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 8.3 MB/s 
     |████████████████████████████████| 352 kB 47.9 MB/s 
     |████████████████████████████████| 1.8 MB 46.4 MB/s 
     |████████████████████████████████| 1.2 MB 49.6 MB/s 
     |████████████████████████████████| 90 kB 10.2 MB/s 
     |████████████████████████████████| 4

In [18]:
print(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py'))

Tensorflow/models/research/object_detection/builders/model_builder_tf2_test.py


In [19]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2021-09-13 09:20:55.710985: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-13 09:20:56.339924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-13 09:20:56.340978: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.7.11: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-09-13 09:20:56.361000: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021

In [20]:
import object_detection

# 2. Download dataset

In [21]:
# Unrar image files from drive and locate in /content/Tensorflow/workspace/images
!unrar x /content/drive/MyDrive/carCoco2017_carPlateKaggleBox4JPG.rar -d /content/Tensorflow/workspace/images

Streaming output truncated to the last 5000 lines.
Extracting  /content/Tensorflow/workspace/images/train/000000447135.xml      82%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000447150.jpg      82%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000447150.xml      82%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000447209.jpg      82%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000447209.xml      82%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000447235.jpg      82%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000447235.xml      82%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000447297.jpg      82%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000447297.xml      82%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000447345.jpg      82%  OK 

# 3. Create Label Map

In [22]:
labels = [{'name':'car', 'id':1},{'name':'licence', 'id':2}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 4. Create TF records

In [23]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

Cloning into 'Tensorflow/scripts'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [24]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


In [25]:
if os.name =='posix':
    !cp {os.path.join(files['PIPELINE_CONFIG_BASE'])} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(files['PIPELINE_CONFIG_BASE'])} {os.path.join(paths['CHECKPOINT_PATH'])}

# 7. Evaluate the Model

In [45]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [46]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [47]:
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=/content/drive/MyDrive/workspaces_tensorflow/eval_model/attempt_20210913_122755_4 --pipeline_config_path=/content/drive/MyDrive/workspaces_tensorflow/eval_model/attempt_20210913_122755_4/pipeline.config --checkpoint_dir=/content/drive/MyDrive/workspaces_tensorflow/eval_model/attempt_20210913_122755_4


In [48]:
!{command}

2021-09-13 09:39:43.455712: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-13 09:39:43.465007: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-13 09:39:43.465833: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
W0913 09:39:43.710820 140043403069312 model_lib_v2.py:1082] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0913 09:39:43.711185 140043403069312 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat

# 10. Zip model and copy to GG Drive

In [50]:
%cd {paths['MODEL_PATH']}

/content/drive/MyDrive/workspaces_tensorflow/eval_model


In [51]:
export_model_path = model_name
export_model_zip = "{}.zip".format(export_model_path)
print(export_model_path)
print(export_model_zip)

attempt_20210913_122755_4
attempt_20210913_122755_4.zip


In [52]:
!zip -r {export_model_zip} {export_model_path}

updating: attempt_20210913_122755_4/ (stored 0%)
updating: attempt_20210913_122755_4/export/ (stored 0%)
updating: attempt_20210913_122755_4/export/checkpoint/ (stored 0%)
updating: attempt_20210913_122755_4/export/checkpoint/ckpt-0.data-00000-of-00001 (deflated 8%)
updating: attempt_20210913_122755_4/export/checkpoint/ckpt-0.index (deflated 80%)
updating: attempt_20210913_122755_4/export/checkpoint/checkpoint (deflated 42%)
updating: attempt_20210913_122755_4/export/saved_model/ (stored 0%)
updating: attempt_20210913_122755_4/export/saved_model/variables/ (stored 0%)
updating: attempt_20210913_122755_4/export/saved_model/variables/variables.data-00000-of-00001 (deflated 8%)
updating: attempt_20210913_122755_4/export/saved_model/variables/variables.index (deflated 78%)
updating: attempt_20210913_122755_4/export/saved_model/assets/ (stored 0%)
updating: attempt_20210913_122755_4/export/saved_model/saved_model.pb (deflated 92%)
updating: attempt_20210913_122755_4/export/pipeline.config (

In [53]:
%cd "/content"

/content


# 11. Notification via Email

In [ ]:
import datetime
import pytz

finish_date = str(datetime.datetime.now(pytz.timezone('Asia/Bangkok')))[:19]
finish_date_short = shorten_time(finish_date)

print(finish_date)
print(finish_date_short)

In [ ]:
sd_year = int(start_date_short[0:4])
sd_month = int(start_date_short[4:6])
sd_day = int(start_date_short[6:8])
sd_hour = int(start_date_short[9:11])
sd_min = int(start_date_short[11:13])
sd_sec = int(start_date_short[13:15])

fd_year = int(finish_date_short[0:4])
fd_month = int(finish_date_short[4:6])
fd_day = int(finish_date_short[6:8])
fd_hour = int(finish_date_short[9:11])
fd_min = int(finish_date_short[11:13])
fd_sec = int(finish_date_short[13:15])

In [ ]:
start = datetime.datetime(year=sd_year, month=sd_month, day=sd_day, hour=sd_hour, minute=sd_min,second=sd_sec)
end = datetime.datetime(year=fd_year, month=fd_month, day=fd_day, hour=fd_hour, minute=fd_min,second=fd_sec)
diff = end - start
print(diff)

In [ ]:
import smtplib
from email.message import EmailMessage

msg_subject = "[GG Colab] Model {}".format(start_date_short)
msg_content = "It take {} for training.".format(diff)

msg = EmailMessage()
msg['Subject'] = msg_subject
msg['From'] = '*'
msg['To']='ptmhoang97@gmail.com'
msg.set_content(msg_content)

server = smtplib.SMTP_SSL('smtp.gmail.com',465)
server.login("ptmhoang97@gmail.com","euclxykrqkkrquqz")
server.send_message(msg)
server.quit()